In [ ]:
from dotenv import load_dotenv
import os 
from azure.core.credentials import AzureKeyCredential
from agent_framework import HostedMCPTool
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient,AzureOpenAIAssistantsClient,AzureOpenAIChatClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential
from agent_framework import ChatMessage


In [ ]:
ms_learn_mcp_tool = HostedMCPTool(
name = "Microsoft Learn MCP Tool",
url = "https://learn.microsoft.com/api/mcp"
)

In [ ]:
async def create_docs_agent():
    credential = AzureCliCredential()
    chat_client = AzureAIAgentClient(
        async_credential=credential,
        project_endpoint = project_client,
        model_deployment_name=model
    )
    try :
        agent = chat_client.create_agent(
            name = "Docs Agent",
            instructions= " You are helpful assistant that helps users find information in Microsoft Learn documentation.",
            tools = [ms_learn_mcp_tool]
        )
        return agent
    finally:
        await credential.close()
        await chat_client.close()

In [ ]:
async def handle_approvals_with_thread(query: str, agent, thread):
    result = await agent.run(query, thread=thread, store=True)
    while len(result.user_input_requests) > 0:
        new_input = []
        for user_input_needed in result.user_input_requests:
            print(
                f"\n⚙️ Function call requested: {user_input_needed.function_call.name}"
                f"\nArguments: {user_input_needed.function_call.arguments}"
            )
            user_approval = input("Approve function call? (y/n): ")
            new_input.append(
                ChatMessage(
                    role="user",
                    contents=[user_input_needed.create_response(user_approval.lower() == "y")],
                )
            )
        result = await agent.run(new_input, thread=thread, store=True)
    return result

In [ ]:
thread = agent.get_new_thread()
query = "Find me information about Azure Functions in Microsoft Learn."
response =await handle_approvals_with_thread(query, agent, thread)

print("\n Assistant Response:", response)